In [10]:
# Márcio Koch - Adaptado de: https://machinelearningmastery.com/pytorch-tutorial-develop-deep-learning-models/
# Pytorch MLP para classificação de multiplas classes
from pandas import read_csv
import pandas as pd
import numpy as np

from numpy import vstack
from numpy import argmax
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch import Tensor
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module

from torch.optim import SGD
from torch.optim import Adam

from torch.nn import CrossEntropyLoss
from torch.nn import NLLLoss
from torch.nn import MSELoss

import torch.nn as nn
import torch as T

from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

import torch.nn.functional as F

labelEncoder = LabelEncoder()
# Definição do dataset
class CSVDataset(Dataset):    
    # Carrega o data set
    # path: caminho do arquivo CSV
    # top: limite de linhas a serem carregadas do dataset, 0 = todas as linhas
    def __init__(self, path, top):        
        # Carrega o CSV como um dataframe, considera a primeira linha como linha de cabeçalho
        df = read_csv(path, header=0, sep=';')

        # Pega as top primeiras linhas do CSV
        if (top > 0):
          df = df.head(top)       

        # Por conveniência, move a coluna com os labels para a última coluna do dataframe
        y = df.pop('FORCE_2020_LITHOFACIES_LITHOLOGY')
        df['FORCE_2020_LITHOFACIES_LITHOLOGY'] = y
       
        # Armazena as entradas e saídas
        # X = entradas da rede
        # y = saídas, classes que a rede prediz
        self.X = df.values[:, :-1]
        self.y = df.values[:, -1]
      
        # Garante que os dados de entrada são floats.
        self.X = self.X.astype('float32')        
       
        # Codifica os labels no estilo binário, a classe predita será bit ligado ("1"), 
        # As demais classes terão bit desligado ("0").
      
        self.y = labelEncoder.fit_transform(self.y)        

    # Retorna o número de linhas no dataset
    def __len__(self):
        return len(self.X)
   
    # Obtém dados de treino e label de uma linha pelo seu índice
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
   
    # Obtém índices para as linhas de treinamento e teste.
    def get_splits(self, n_test=0.3):        
        # Determina os tamanhos do conjunto de teste e treino
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size

        # Divide as amostras de dados a serem utilizadas
        return random_split(self, [train_size, test_size])

# Definição do modelo da rede neural
class MLP(Module):

    # Define os elementos do modelo
    def __init__(self, n_inputs):
      super(MLP, self).__init__()
        
      self.fc1 = nn.Linear(n_inputs, 50)
      self.fc2 = nn.Linear(50, 40)
      self.fc3 = nn.Linear(40, 30)
      self.fc4 = nn.Linear(30, 20)
      self.fc5 = nn.Linear(20, 12)     

      #20% probability Dropout 
      #self.dropout = nn.Dropout(p=.2)
   
    # Entrada para o "forward propagate"
    def forward(self, x):
       #x = F.relu(self.fc1(x))
       #x = F.relu(self.fc2(x))
       #x = F.relu(self.fc3(x))
       #x = F.relu(self.fc4(x))

       x = T.tanh(self.fc1(x))
       x = T.tanh(self.fc2(x))
       x = T.tanh(self.fc3(x))
       x = T.tanh(self.fc4(x))

       x = self.fc5(x)
       return x

# Prepara o dataset
def prepare_data(path, top):    
    # Carrega o dataset
    dataset = CSVDataset(path, top)
    
    # Calcula a divisão para treino e teste
    train, test = dataset.get_splits()

    # Prepara os carregadores de dados em batch
    train_dl = DataLoader(train, batch_size=32, shuffle=True)    
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)

    return train_dl, test_dl

# Avalia o modelo
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):        
        # Avalia o modelo no conjunto de teste
        yhat = model(inputs)
        
        # Retorna o array numpy
        yhat = yhat.detach().numpy()        
        actual = targets.numpy()
       
        # Converte para rótulos de classe
        yhat = argmax(yhat, axis=1)
        
        # Remodela para empilhamento
        actual = actual.reshape((len(actual), 1))
        yhat = yhat.reshape((len(yhat), 1))
       
        # Armazena na lista de predições
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    
    # Calcula a acurrácia
    acc = accuracy_score(actuals, predictions)
    return acc

# Faz a predição da classe de uma data linha de dados desconhecidos
def predict(row, model):    
    # Converte o conteúdo da linha para dados
    row = Tensor([row])      

    # Faz a predição
    yhat = model(row)   
    
    # Retorna o array numpy
    yhat = yhat.detach().numpy()
    return yhat

# Faz as predições do dataset hidden e salva o resultado num arquivo CSV
def predic_hidden(model, top):
  path = '/content/drive/MyDrive/Furb/hidden.csv'
  dfh = read_csv(path, header=0, sep=';')
  
  if (top == 0):
    top = len(dfh)
  hiddens = []

  for i in range(0, top):   
    row = dfh.iloc[i].values
    yhat = predict(row, model)
    yy = argmax(yhat)    

    predictedLabel = str(int(labelEncoder.inverse_transform([yy])))    
    hiddens.append(predictedLabel)   

  np.savetxt("hidden_result.csv", 
           hiddens,
           delimiter =",", 
           fmt ='% s')
  
  print('DONE predic_hidden')
  
#############################################################################  
# Treina o modelo
def train_model(train_dl, model, epocas):
    erros = 0    
    # Define a estratégia de otimização
    criterion = CrossEntropyLoss()    
    
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    #optimizer = Adam(model.parameters(), lr=0.003)

    # Enumera/itera pelas épocas    
    for epoch in range(epocas):
        print('Fazendo EPOCA:', epoch)
        # Enumera/itera pelos mini batches
        for i, (inputs, targets) in enumerate(train_dl):            
            # Limpa os gradientes
            optimizer.zero_grad()
            
            # Computa o modelo de saída
            yhat = model(inputs)
            
            # Calcula a perda (loss)
            try:
              loss = criterion(yhat, targets)
             
              # Faz o back propagation
              loss.backward()
             
              # Atualiza os pesos do modelo
              optimizer.step()
            except  Exception as inst:
              erros = erros + 1
              print('Erro: ', inst);
              print('targets: ', targets);

    print('Total erros:', erros)
#############################################################################      

# Prepara os dados
path = '/content/drive/MyDrive/Furb/lithology.csv'

# Quantidade de linhas do dataset a serem utilizadas, 0 = todas
top = 0

# Quantidade de épocas de treinamento
epocas = 300

train_dl, test_dl = prepare_data(path, top)
print(len(train_dl.dataset), len(test_dl.dataset))

# Define a rede, quantidade de variáveis de entrada da rede
model = MLP(30)

# Treina o modelo
train_model(train_dl, model, epocas)

# Avalia o modelo
acc = evaluate_model(test_dl, model)
print('Accuracy: %.3f' % acc)

print('DONE train')

#predic_hidden(model, 0)

819358 351153
Fazendo EPOCA: 0
Fazendo EPOCA: 1
Fazendo EPOCA: 2
Fazendo EPOCA: 3
Fazendo EPOCA: 4
Fazendo EPOCA: 5
Fazendo EPOCA: 6
Fazendo EPOCA: 7
Fazendo EPOCA: 8
Fazendo EPOCA: 9
Fazendo EPOCA: 10
Fazendo EPOCA: 11
Fazendo EPOCA: 12
Fazendo EPOCA: 13
Fazendo EPOCA: 14
Fazendo EPOCA: 15
Fazendo EPOCA: 16
Fazendo EPOCA: 17
Fazendo EPOCA: 18
Fazendo EPOCA: 19
Fazendo EPOCA: 20
Fazendo EPOCA: 21
Fazendo EPOCA: 22
Fazendo EPOCA: 23
Fazendo EPOCA: 24
Fazendo EPOCA: 25
Fazendo EPOCA: 26
Fazendo EPOCA: 27
Fazendo EPOCA: 28
Fazendo EPOCA: 29
Fazendo EPOCA: 30
Fazendo EPOCA: 31
Fazendo EPOCA: 32
Fazendo EPOCA: 33
Fazendo EPOCA: 34
Fazendo EPOCA: 35
Fazendo EPOCA: 36
Fazendo EPOCA: 37
Fazendo EPOCA: 38
Fazendo EPOCA: 39
Fazendo EPOCA: 40
Fazendo EPOCA: 41
Fazendo EPOCA: 42
Fazendo EPOCA: 43
Fazendo EPOCA: 44
Fazendo EPOCA: 45
Fazendo EPOCA: 46
Fazendo EPOCA: 47
Fazendo EPOCA: 48
Fazendo EPOCA: 49
Fazendo EPOCA: 50
Fazendo EPOCA: 51
Fazendo EPOCA: 52
Fazendo EPOCA: 53
Fazendo EPOCA: 54
Fazend

In [11]:
predic_hidden(model, 0)

DONE predic_hidden
